In [1]:
# pip install xlrd

In [2]:
from bs4 import BeautifulSoup as bs 
from urllib import request
import requests
import time 
import folium
import re
import pandas as pd
import folium

In [3]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        return getLatLng(fixed_addr)

def add_marker(mapobj, x, y):
    folium.Marker((y,x), icon=folium.Icon(color='green', icon='glyphicon glyphicon-print')).add_to(mapobj)

def get_avg_point(point_list):
    x = sum(map(lambda i:float(i[0]), point_list)) / len(point_list)
    y = sum(map(lambda i:float(i[1]), point_list)) / len(point_list)
    return y,x

In [14]:
url2 = 'http://www.minwon.go.kr/main?d=AA100DataDownApp&SAVE_FILE_NM=38800.xls'
file = 'noman.xls'
request.urlretrieve(url2, file)

('noman.xls', <http.client.HTTPMessage at 0x137dbc704c8>)

In [27]:
df_noman = pd.read_excel('D:/bigdata/Python_workspace/jupyter/project/noman.xls')
addr_point_list = []
for addr in df_noman['도로명주소']:
    addr_point_list.append(getLatLng(addr))

In [95]:
def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    cnt = 0
    disabled_sup = list(set(df_noman['장애인편의발급기형태']))
    for x, y in addr_point_list:
        if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
            add_marker(map_noman, x, y)
        cnt += 1
        if cnt == 100:
            break
    return map_noman


In [ ]:
input_addr = input('도로명을 입력하세요: ')
# print(input_addr)
addr_pattern = re.compile(f'{input_addr}')
df_noman_road = df_noman.sort_index(by='도로명주소')
start = -1
end = -1
for i in range(len(df_noman_road['도로명주소'])):
#     print(i)
#     print(df_noman_road['도로명주소'][i])
    if re.search(addr_pattern, df_noman_road['도로명주소'][i]):
        if start != -1:
            continue
        start = i
    else:
        if start == -1:
            continue
        end = i
        break
# print(start, end)
# df_noman['도로명주소'][start:end]
# df_noman['도로명주소'][0]
df_noman_road['도로명주소'][1]